In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
import sys
sys.path.append('../script')
from plots import *
from data_extraction import *

In [4]:
#as always lets load the data
conn = connection()

In [5]:
data = table_to_sql(conn, 'telecom_data')

c:\Users\ok\Desktop\10 Acadamy\week_1\week_1\notebook\../script\data_extraction.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)


In [6]:
data.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,1970-01-01,4/25/2019 14:35,1970-01-01,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,1970-01-01,4/25/2019 8:15,1970-01-01,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1970-01-01,4/25/2019 11:58,1970-01-01,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,1970-01-01,4/25/2019 7:36,1970-01-01,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,1970-01-01,4/25/2019 10:40,1970-01-01,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [7]:
#first lets start with creating a data with the newwork features that we need to analyze the user exeperience
experiance = data[['Bearer Id', 'MSISDN/Number', 'Avg RTT DL (ms)','Avg RTT UL (ms)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)', 'Handset Type']]

experiance.head()

,Bearer Id,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Handset Type
0,1.311448e+19,3.366496e+10,42.000000,5.000000,2.080991e+07,759658.664811,23.0,44.0,Samsung Galaxy A5 Sm-A520F
1,1.311448e+19,3.368185e+10,65.000000,5.000000,2.080991e+07,759658.664811,16.0,26.0,Samsung Galaxy J5 (Sm-J530)
2,1.311448e+19,3.376063e+10,109.795706,17.662883,2.080991e+07,759658.664811,6.0,9.0,Samsung Galaxy A8 (2018)
3,1.311448e+19,3.375034e+10,109.795706,17.662883,2.080991e+07,759658.664811,44.0,44.0,undefined
4,1.311448e+19,3.369980e+10,109.795706,17.662883,2.080991e+07,759658.664811,6.0,9.0,Samsung Sm-G390F


In [8]:
#now lets find the total of these newtork features for each user
experiance['total_RTT'] = experiance['Avg RTT DL (ms)'] + experiance['Avg RTT UL (ms)']
experiance['total_TCP'] = experiance['TCP DL Retrans. Vol (Bytes)'] + experiance['TCP UL Retrans. Vol (Bytes)']
experiance['total_TP'] = experiance['Avg Bearer TP DL (kbps)'] + experiance['Avg Bearer TP UL (kbps)']

experiance.head()

C:\Program Files\KMSpico\temp\ipykernel_19376\2372917508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experiance['total_RTT'] = experiance['Avg RTT DL (ms)'] + experiance['Avg RTT UL (ms)']
C:\Program Files\KMSpico\temp\ipykernel_19376\2372917508.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experiance['total_TCP'] = experiance['TCP DL Retrans. Vol (Bytes)'] + experiance['TCP UL Retrans. Vol (Bytes)']
C:\Program Files\KMSpico\temp\ipykernel_19376\2372917508.py:4: SettingWithCopyWarning: 
A valu

,Bearer Id,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Handset Type,total_RTT,total_TCP,total_TP
0,1.311448e+19,3.366496e+10,42.000000,5.000000,2.080991e+07,759658.664811,23.0,44.0,Samsung Galaxy A5 Sm-A520F,47.000000,2.156957e+07,67.0
1,1.311448e+19,3.368185e+10,65.000000,5.000000,2.080991e+07,759658.664811,16.0,26.0,Samsung Galaxy J5 (Sm-J530),70.000000,2.156957e+07,42.0
2,1.311448e+19,3.376063e+10,109.795706,17.662883,2.080991e+07,759658.664811,6.0,9.0,Samsung Galaxy A8 (2018),127.458589,2.156957e+07,15.0
3,1.311448e+19,3.375034e+10,109.795706,17.662883,2.080991e+07,759658.664811,44.0,44.0,undefined,127.458589,2.156957e+07,88.0
4,1.311448e+19,3.369980e+10,109.795706,17.662883,2.080991e+07,759658.664811,6.0,9.0,Samsung Sm-G390F,127.458589,2.156957e+07,15.0


In [31]:
#now lets find the aggregat  of these features for each user

agg_experiance = experiance.groupby('MSISDN/Number').agg({'total_RTT': 'mean', 'total_TCP': 'mean', 'total_TP': 'mean', 'Handset Type': 'first'})
agg_experiance.rename(columns={'total_RTT': 'avg_RTT', 'total_TCP': 'avg_TCP', 'total_TP': 'avg_TP'}, inplace=True)
agg_experiance.head()

,avg_RTT,avg_TCP,avg_TP,Handset Type
MSISDN/Number,,,,
3.360100e+10,46.000000,2.156957e+07,76.0,Huawei P20 Lite Huawei Nova 3E
3.360100e+10,31.000000,2.156957e+07,99.0,Apple iPhone 7 (A1778)
3.360100e+10,127.458589,2.156957e+07,97.0,undefined
3.360101e+10,84.000000,7.607247e+05,248.0,Apple iPhone 5S (A1457)
3.360101e+10,59.500000,1.547020e+07,28422.0,Apple iPhone Se (A1723)


In [29]:
#lets find the top, and bottom 10 average RTT values
top_10_RTT = agg_experiance.sort_values(by='avg_RTT', ascending=False)
print('The top 10 values of RTT per user are: \n', top_10_RTT['avg_RTT'].head(10))
print('---------------------------------------------------')
print('The bottom to values of RTT per user are: \n',top_10_RTT['avg_RTT'].tail(10))
print('---------------------------------------------------')
freq = agg_experiance['avg_RTT'].value_counts()
print('The most frequent values of RTT are: \n', freq.head(10))

The top 10 values of RTT per user are: 
 MSISDN/Number
3.366232e+10    96924.0
3.368369e+10    54848.0
3.366087e+10    32335.0
3.376094e+10    27278.0
3.360679e+10    26300.0
3.367182e+10    25715.0
3.378187e+10    25388.0
3.378401e+10    24738.0
3.369855e+10    23010.5
3.362963e+10    20980.0
Name: avg_RTT, dtype: float64
---------------------------------------------------
The bottom to values of RTT per user are: 
 MSISDN/Number
3.363468e+10    10.0
3.361085e+10     9.0
3.367220e+10     9.0
3.364768e+10     8.0
3.366862e+10     6.0
3.365949e+10     5.0
3.369505e+10     4.0
3.366020e+10     2.0
3.365298e+10     0.0
3.366158e+10     0.0
Name: avg_RTT, dtype: float64
---------------------------------------------------
The most frequent values of RTT are: 
 127.458589    18514
29.000000      2950
39.000000      2367
38.000000      1639
40.000000      1583
30.000000      1577
28.000000      1456
49.000000      1308
41.000000      1207
31.000000      1190
Name: avg_RTT, dtype: int64


In [34]:
#lets find the top, and bottom 10 average TP values
top_10_TP = agg_experiance.sort_values(by='avg_TP', ascending=False)
print('The top 10 values of TP per user are: \n', top_10_TP['avg_TP'].head(10))
print('---------------------------------------------------')
print('The bottom to values of TP per user are: \n',top_10_TP['avg_TP'].tail(10))
print('---------------------------------------------------')
freq = agg_experiance['avg_TP'].value_counts()
print('The most frequent values of TP are: \n', freq.head(10))

The top 10 values of TP per user are: 
 MSISDN/Number
3.365991e+10    283931.0
3.366268e+10    265724.0
3.366566e+10    265176.5
3.366349e+10    254950.0
3.362795e+10    238982.0
3.362118e+10    235365.0
3.378240e+10    230982.0
3.365148e+10    228303.0
3.376033e+10    227334.0
3.376267e+10    226505.0
Name: avg_TP, dtype: float64
---------------------------------------------------
The bottom to values of TP per user are: 
 MSISDN/Number
3.363143e+10    0.0
3.367247e+10    0.0
3.363144e+10    0.0
3.366560e+10    0.0
3.363306e+10    0.0
3.367056e+10    0.0
3.365873e+10    0.0
3.363359e+10    0.0
3.363392e+10    0.0
3.360889e+10    0.0
Name: avg_TP, dtype: float64
---------------------------------------------------
The most frequent values of TP are: 
 15.0    2872
63.0    2138
97.0    1094
90.0    1062
98.0    1025
96.0     938
99.0     918
89.0     881
91.0     864
95.0     856
Name: avg_TP, dtype: int64


In [35]:
#lets find the top, and bottom 10 average RTT values
top_10_tcp = agg_experiance.sort_values(by='avg_TCP', ascending=False)
print('The top 10 values of TCP per user are: \n', top_10_tcp['avg_TCP'].head(10))
print('---------------------------------------------------')
print('The bottom to values of TCP per user are: \n',top_10_tcp['avg_TCP'].tail(10))
print('---------------------------------------------------')
freq = agg_experiance['avg_TCP'].value_counts()
print('The most frequent values of TCP are: \n', freq.head(10))

The top 10 values of TCP per user are: 
 MSISDN/Number
3.369858e+10    4.301477e+09
3.365871e+10    4.288121e+09
3.366491e+10    4.268647e+09
3.365863e+10    4.254660e+09
3.366613e+10    4.211258e+09
3.366877e+10    4.166595e+09
3.366131e+10    4.137938e+09
3.366240e+10    4.117805e+09
3.366682e+10    3.968121e+09
3.376264e+10    3.786871e+09
Name: avg_TCP, dtype: float64
---------------------------------------------------
The bottom to values of TCP per user are: 
 MSISDN/Number
3.369969e+10    182.0
3.367937e+10    179.0
3.378420e+10    177.0
3.376964e+10    176.0
3.365068e+10    176.0
3.360783e+10    143.0
3.362636e+10    134.0
3.376206e+10    129.0
3.361478e+10    128.0
3.365013e+10     97.0
Name: avg_TCP, dtype: float64
---------------------------------------------------
The most frequent values of TCP are: 
 2.156957e+07    55974
2.081121e+07      367
2.081124e+07      167
7.609887e+05      145
2.119039e+07       82
2.081123e+07       81
7.623187e+05       78
7.597507e+05       7

In [36]:
#most frequent handsets
freq = agg_experiance['Handset Type'].value_counts()
print('The most frequent handsets are: \n', freq.head(10))

The most frequent handsets are: 
 Huawei B528S-23A                10615
Apple iPhone 6S (A1688)          6759
undefined                        6669
Apple iPhone 6 (A1586)           6260
Apple iPhone 7 (A1778)           4699
Apple iPhone Se (A1723)          3754
Apple iPhone 8 (A1905)           3543
Samsung Galaxy S8 (Sm-G950F)     3245
Apple iPhone Xr (A2105)          3072
Samsung Galaxy J5 (Sm-J530)      2748
Name: Handset Type, dtype: int64


In [40]:
#4.3.1 average throuput per handset type
avg_TP_by_handset_type=experiance.groupby('Handset Type').agg({'total_TP':'mean'})
avg_TP_by_handset_type.rename(columns={'total_TP':'avg_tp'},inplace=True)
avg_TP_by_handset_type.head()

,avg_tp
Handset Type,
A-Link Telecom I. Cubot A5,23510.0
A-Link Telecom I. Cubot Note Plus,6699.0
A-Link Telecom I. Cubot Note S,8937.0
A-Link Telecom I. Cubot Nova,56217.0
A-Link Telecom I. Cubot Power,69468.0


In [41]:
#4.3.2 average TCP per handset type
avg_TCP_by_handset_type=experiance.groupby('Handset Type').agg({'total_TCP':'mean'})
avg_TCP_by_handset_type.rename(columns={'total_TCP':'avg_tcp'},inplace=True)
avg_TCP_by_handset_type.head()

,avg_tcp
Handset Type,
A-Link Telecom I. Cubot A5,2.156957e+07
A-Link Telecom I. Cubot Note Plus,6.162130e+05
A-Link Telecom I. Cubot Note S,4.141173e+07
A-Link Telecom I. Cubot Nova,1.371700e+05
A-Link Telecom I. Cubot Power,8.041000e+03


In [44]:
agg_experiance

,avg_RTT,avg_TCP,avg_TP,Handset Type
MSISDN/Number,,,,
3.360100e+10,46.000000,2.156957e+07,76.0,Huawei P20 Lite Huawei Nova 3E
3.360100e+10,31.000000,2.156957e+07,99.0,Apple iPhone 7 (A1778)
3.360100e+10,127.458589,2.156957e+07,97.0,undefined
3.360101e+10,84.000000,7.607247e+05,248.0,Apple iPhone 5S (A1457)
3.360101e+10,59.500000,1.547020e+07,28422.0,Apple iPhone Se (A1723)
...,...,...,...,...
3.379000e+10,52.000000,2.180450e+05,10365.0,Huawei Honor 9 Lite
3.379000e+10,40.000000,2.156957e+07,116.0,Apple iPhone 8 Plus (A1897)
3.197021e+12,127.458589,2.156957e+07,1.0,Quectel Wireless. Quectel Ec25-E


In [48]:
#K-mean clastering to claster users according to their experiance
agg_experiance.drop(['MSISDN/Number'], inplace= True, axis=1)

KeyError: "['MSISDN/Number'] not found in axis"

Task 4. 1 - Aggregate, per customer, the following information (treat missing & outliers by replacing by the mean or the mode of the corresponding variable):
●	Average TCP retransmission
●	Average RTT
●	Handset type
●	Average throughput
Task 4.2 - Compute & list 10 of the top, bottom and most frequent:
a.	TCP values in the dataset. 
b.	RTT values in the dataset.
c.	Throughput values in the dataset.
Task 4.3 - Compute & report:
d.	The distribution of the average throughput  per handset type and provide interpretation for your findings.
e.	The average TCP retransmission view per handset type and provide interpretation for your findings.
Task 4.4 - Using the experience metrics above, perform a k-means clustering (where k = 3) to segment users into groups of experiences and provide a brief description of each cluster. (The description must define each group based on your understanding of the data)